In [35]:
import sys
import os
from glob import glob
import pandas as pd
from PIL import Image
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
sys.path.insert(0, '../')
from model import VanillaResNet
from transform_settings import configure_transform

In [9]:
transform = configure_transform('train', 'base')

In [32]:
model = VanillaResNet()

In [24]:
class TestDataset(Dataset):
    def __init__(self, data_root, transform=None):
        self.img_paths = glob(os.path.join(data_root, '*'))
        self.transform = transform

    def __getitem__(self, index):
        name = os.path.basename(self.img_paths[index])
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        return name, image

    def __len__(self):
        return len(self.img_paths)



In [25]:
dataset = TestDataset(data_root='../input/data/eval/images', transform=transform)

In [27]:
loader = DataLoader(dataset, batch_size=1, shuffle=False, drop_last=False)

In [40]:
name = name[0]

In [41]:
name

'db88dee46ba3f5fe21d16b345ed8c4d3c6df2281.jpg'

In [38]:
pred_label = F.softmax(model(img), dim=1).argmax().item()

In [42]:
pred_dict = {}

In [46]:
pred_dict[name] = pred_label

In [53]:
info = pd.read_csv('../input/data/eval/info.csv')

In [55]:
info.merge(pred_df, how='left', on='ImageID')

,ImageID,ans_x,ans_y
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,NaN
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,NaN
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,NaN
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,NaN
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,NaN
...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,NaN
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,NaN
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,NaN
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,NaN


In [51]:
pred_df = pd.Series(pred_dict).to_frame('ans')
pred_df = pred_df.reset_index().rename({'index':'ImageID'}, axis=1)
pred_df

,ImageID,ans
0,db88dee46ba3f5fe21d16b345ed8c4d3c6df2281.jpg,17
